In [1]:
!pip install openai

from openai import OpenAI
import networkx as nx

import numpy as np
from scipy import stats as sps
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
np.random.seed(42)

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pickle
with open("data/Moviegroups.pkl", "rb") as f:
    movie_group = pickle.load(f)
with open("data/Movietags.pkl", "rb") as f:
    movietags = pickle.load(f)

In [3]:
with open("data/extended_tags.pkl", "rb") as f:
    extended_tags = pickle.load(f)

In [4]:
with open("data/sampled_split_tags.pkl", "rb") as f:
    sampled_split_tags = pickle.load(f)

In [5]:
OPENAI_API_KEY="..." # from openrouter

In [6]:
import time
import numpy as np

class Agent:
    def __init__(self, api_key, model, description=None, api_type="Groq", max_retries=5, retry_delay=50):
        self.client = OpenAI(api_key=api_key, base_url="https://openrouter.ai/api/v1")
        self.model = model
        self.description = description
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.history = [
            {"role": "system", "content": f"{self.description}"}
        ]

    def _generate_response(self, messages):
        print(f"# call with {messages}")
        response = None
        try:
          response = self.client.chat.completions.create(
              messages=messages,
              model=self.model,
              temperature=0,
              top_p=0.5
          )
        except Exception as e:
          print(f"# Error: {e}")
        print(f"# raw: {response}")

        # Check for API error
        if getattr(response, "error", None):
            print(f"# API error {response.error.get('code')}: {response.error.get('message')}")
            return None

        # Check if choices exist
        if not getattr(response, "choices", None):
            print("# No choices returned")
            return None

        content = getattr(response.choices[0].message, "content", None)
        if not content:
            print("# Empty content returned")
            return None

        return content

    def stateful_chat(self, user_message, history_message = ""):
        self.history.append({"role": "user", "content": user_message})
        response = None
        for attempt in range(self.max_retries):
            response = self._generate_response(self.history)
            if response is not None:
                try:
                    tmp = [float(x) for x in response.split()]
                    if len(tmp) != 3:
                        raise ValueError("Response does not contain exactly 3 numbers")
                except ValueError:
                    # Parsing failed or wrong number of floats -> retry
                    print(f"# Retry {attempt+1}/{self.max_retries} after {self.retry_delay}s...")
                    time.sleep(self.retry_delay)
                    continue  # go to next retry attempt

                # If we got here, parsing succeeded
                print(f"#{response}")
                self.history.pop()
                if history_message != "":
                    self.history.append({"role": "user", "content": history_message})
                return response
            print(f"# Retry {attempt+1}/{self.max_retries} after {self.retry_delay}s...")
            time.sleep(self.retry_delay)

        self.history.pop()
#         self.history.append({"role": "user", "content": history_message})
        return response

    def reset_history(self):
        self.history = [self.history[0]]

### Likelihood collection

In [7]:
top_n = 3

In [8]:
llm_agent = Agent(
    api_key=OPENAI_API_KEY,
    model="openai/gpt-oss-120b",
    api_type="OpenAI",
    description="Presume that any of the three movies is initially equally likely. Then a single tag is given, related to the correct movie. Return ONLY three space-separated decimal numbers. NO explanations, NO formatting.",
)

In [10]:
with open("data/Likelihoods", "rb") as f:
    id_likelihood_precomputed = np.load(f)

In [11]:
id_likelihood = np.zeros((100, len(extended_tags), 3))

In [12]:
id_likelihood = np.copy(id_likelihood_precomputed)

In [13]:
num_samples = 11

In [16]:
for sample in range(num_samples):
    llm_agent.reset_history()
    for clue_id in range(len(extended_tags)):
        clue = extended_tags[clue_id]
        if (clue_id >= 2058 + 140):
            continue
        if id_likelihood[sample][clue_id].sum() == 0:
            head_prompt = f"""Given the tag: \"{clue}\", what is your belief over the three classes [Pulp Fiction, Forrest Gump, Fight Club]? Return ONLY EXACTLY three space-separated decimal numbers. NO explanations, NO text, NO formatting."""
            response = llm_agent.stateful_chat(head_prompt)
            id_likelihood[sample][clue_id] = np.array([float(x) for x in response.split()])
#         print(f"clue {clue_id}, {clue}: {id_likelihood[sample][clue_id]}")

In [17]:
with open("data/Likelihoods", "rb") as f:
    id_likelihood = np.load(f)